# Huggingface Transformers로 모델 이해하기 - BERT
## 여기서 배울 수 있는 것
- Transformer Architecture의 encoder block에 대한 이해
- `transformers.BertModel` 클래스에 대한 이해

## 개요
[BERT](https://arxiv.org/abs/1810.04805)는  Transformer의 Encoder Block들을 MLM(Masked Language Model) + NSP(Next Sentence Prediction)의 학습전략을 활용하여 Bidirectional하게 학습한 Self-Supervised Learning 모델이다.

BERT가 나온 이후로 NLP의 패러다임은 완전히 뒤바뀌었다. 기존의 word2vec, fasttext와 같은 단어 수준의 임베딩 기법들은 BERT와 같은 문장 수준의 임베딩 기법들로, RNN 계열의 모델 아키텍쳐는 Transformer로 완전히 대체되었다. 이와 함께 NLP 연구가 모델은 더 크게, 데이터와 GPU는 더 많이 사용하는 이른바 군비경쟁과 비슷한 상황으로 나아가고 있다.

## 모델 아키텍쳐

### Overview
*bert-base*

<img src="https://blog.rasa.com/content/images/2019/09/BERT-components-1.png" width=800>

- 모델의 아키텍쳐를 다음과 같이 나눠볼 수 있음.
    - 1) **Embedding Layer**
    - 2) **Encoder Layers**
        - Transformer Encoder blocks
    - 3) **Pooler**
        - Bert's Hidden State Output
- 논문에서 Bert는 base, large로 구분됨.
    - Bert-base (GPT와 동일 사이즈)
        - L=12 (transformer encoder block), H=768 (Hidden size), A=12 (Number of head MultiAttention)
    - Bert-large 
        - L=24, H=1024, A=16

#### `transformers.modeling_bert.BertModel`
- [monologg님의 hanbert를 사용함.](https://github.com/monologg/HanBert-Transformers)

In [1]:
# import modules
import torch
import torch.nn as nn
from transformers import BertModel
from models.tokenization_hanbert import HanBertTokenizer

In [2]:
# set model and tokenizer
model_name_or_path = 'models/HanBert-54kN-torch' # models아래에 PLM을 저장함.
model = BertModel.from_pretrained(model_name_or_path)
tokenizer = HanBertTokenizer.from_pretrained(model_name_or_path)

In [13]:
# # If you want to view the model uncomment this
# model 

### 1) BertEmbeddings
<img src="https://user-images.githubusercontent.com/1250095/50039788-8e4e8a00-007b-11e9-9747-8e29fbbea0b3.png">

- Sequence와 Sentence 차이
    - 본 논문에서 **Sequence** Bert의 input으로 들어가는 token들의 sequence를 의미함.
    - 본 논문에서 **Sentence**는 임의의 연속된 텍스트를 의미함. (언어학적 의미의 sentence와 좀 다름)
    - e.g.) input pair (Question, Answer) $\Longrightarrow$ 1 sequence, 2 sentence
- Input Representation
    - 다양한 down-stream task를 일관되게 처리하기 위한 구조로 만들어짐. (Single Sentence, Pair of Sentences를 동시에 받아야 됨.)
    - `[SEP]` token으로 Sentence를 구분함. Segment embedding으로 sequence 구분을 임베딩함.
    - $input\_embedding = token\_embedding + segment\_embedding + position\_embedding$

#### `transformers.modeling_bert.BertEmbeddings`
```python
class BertEmbeddings(nn.Module):
    """Construct the embeddings from word, position and token_type embeddings.
    """
    def __init__(self, config):
        super().__init__()
        # token Embedding, 기존의 단어 수준의 임베딩과 동일함. 단어에 대하여 context-independent한 의미를 학습함.
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        
        # position Embedding, sentence position
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        
        # token type Embedding, Segment embedding, 문장 pair의 수
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)
        
        # torch.nn.LayerNorm
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        
        # dropout hidden_dropout_prob = 0.1
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids=None, token_type_ids=None, position_ids=None, inputs_embeds=None):
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]
        device = input_ids.device if input_ids is not None else inputs_embeds.device
        
        # position의 경우 sequence 길이만큼 인덱스를 만들어줌.
        if position_ids is None:
            position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).expand(input_shape)
        
        # token_type_ids가 따로 없는 경우(e.g. RoBERTa) 사실상 없는 코드
        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)
        
        # 임베딩을 직접 넣어주면 굳이 input embedding을 안뽑는다.
        if inputs_embeds is None:
            inputs_embeds = self.word_embeddings(input_ids)
        
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = inputs_embeds + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings
```

In [69]:
# hanbert의 경우 vocab_size 54000, hidden_size 768
model.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(54000, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

- input sequence 예제 데이터
    - single sentence
    - `bs=4`

In [71]:
# sample data 
with open('data/sample.txt') as f:
    docs = f.read().split('\n') 
print(f"batch size {len(docs)}")
print(docs)

batch size 4
['싼맛에 마트에서 사봤어요. 수분크림 찾기가 힘들었는데 별 기대않고 썼는데 건조하지도 않고 부담되지도 않고 좋더라구요. 싸니까 퍽퍽 바르게 되구요. 2통째 쓰고 있습니다.', '정말 쏘쏘한 제품....가성비는 정말 좋긴 하지만 특별히 좋다는 느낌이 전혀 없어요... 보니까 수부지용 크림인데 수부지용인것치고는 속보습을 잡아준다는 느낌도 특별히 없고 그냥 저렴한데 용량 많은 제품인거 같아요ㅎㅎ', '트러블 일어나지 않을까 걱정했지만 그런 걱정이 필요없는 제품. 세안하고 나서 바르면 촉촉하고, 메이크업 전에 바르면 화장이 더 잘 먹어요', '진짜 너무 실망이에요 ;; 일리윤은 믿고 쓰는 편이였는데 같은 라인이 맞나 싶을 정도로 당황스러웠네요. 우선 뚜껑 열자마자 엄마 화장품 냄새? 정말 역하게 훅 올라와서 불쾌했구요. 제형은 크림이라기보단 젤에 가까운것같은데 건성 피부 기준 너무너무 건조했고.. 그 주제에 밀리기까지 하네요 ㅋㅋㅋㅋㅋㅋ']


- `max_length = 64`
- BERT는 absolute position embedding이 있기 때문에 오른쪽 패딩.
    > BERT is a model with absolute position embeddings so it’s usually advised to pad the inputs on the right rather than the left.

In [80]:
# batch encoding
batch_encoding = tokenizer.batch_encode_plus(docs, max_length=64, pad_to_max_length=True)
input_ids = torch.tensor(batch_encoding['input_ids'], dtype=torch.long)
token_type_ids = torch.tensor(batch_encoding['token_type_ids'], dtype=torch.long)
attention_mask = torch.tensor(batch_encoding['attention_mask'], dtype=torch.long)

- input embedding

In [82]:
embeddings = model.embeddings(input_ids=input_ids, token_type_ids=token_type_ids)
embeddings.shape #(batch_size, sequence_length, embedding_size)

torch.Size([4, 64, 768])

### 2) BertEncoder

- BertEncoder는 기본적으로 `BertLayer`를 `L`개를 반복문을 통해서 쌓아주는 구조다. 

```python
self.layer = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])
```

### 2-1) BertLayer

In [90]:
model.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [90]:
model.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [91]:
model.encoder.layer[0].attention

BertAttention(
  (self): BertSelfAttention(
    (query): Linear(in_features=768, out_features=768, bias=True)
    (key): Linear(in_features=768, out_features=768, bias=True)
    (value): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (output): BertSelfOutput(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [92]:
model.encoder.layer[0].attention.self

BertSelfAttention(
  (query): Linear(in_features=768, out_features=768, bias=True)
  (key): Linear(in_features=768, out_features=768, bias=True)
  (value): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

### BertPooler

BertEmbeddings(
  (word_embeddings): Embedding(54000, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

- Architecture 설명.
    - embedding
    - encoder part
        - bert layer (bertstyle transformer block)
            - sub modules.
        - bert pooler

In [6]:
model.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)